In [2]:
get_ipython().magic('matplotlib notebook')
get_ipython().magic('load_ext autoreload')
get_ipython().magic('autoreload 2')
import sys
import numpy as np
import time
import matplotlib.pyplot as plt
import copy as cp

#+____IMPORT FESOM RELATET ROUTINES____________________________________________+
from set_inputarray  import *
from sub_fesom_mesh  import * 
from sub_fesom_mesh  import * 
from sub_fesom_data  import * 
from sub_fesom_plot  import *
from colormap_c2c    import *


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
#+_____________________________________________________________________________+
#|                                                                             |
#|                         *** LOAD FVSOM MESH ***                             |
#|                                                                             |
#+_____________________________________________________________________________+
# for more options look in set_inputarray.py
inputarray=set_inputarray()
inputarray['save_fig'        ] = False
inputarray['save_figpath'    ] = '/home/ollie/pscholz/figures/poster_TRR181_winterschool/'


# set plot box for cyl projection (default: [-180,180,-90,90])
#inputarray['which_box'] = [0,360,-90,90] ; inputarray['mesh_focus']=180
inputarray['which_box'] = [-180,180,-90,90]
dx=5.0
dy=20.0 
inputarray['which_box'] = [0.0-dx,0.0+dx,70.0-dy,70.0+dy]

# set projection variable --> the lon/lat projection plot ranges are set via 
# inputarray['which_box'] = [lonmin,lonmax,latmin,latmax]
inputarray['proj'     ] = 'cyl' # 'ortho', 'cyl', 'npstere' , 'spstere'
inputarray['proj_lon' ] = -45 #only for ortho
inputarray['proj_lat' ] = 45 #only for ortho

#inputarray['mesh_id'],inputarray['mesh_dir'] = 'COREv2','/work/ollie/pscholz/mesh_fesom2.0/mesh_CORE2_dsidorenko_meanval/'
inputarray['mesh_id'],inputarray['mesh_dir'], inputarray['mesh_rotate']= 'lofoten','/work/ollie/pscholz/mesh_fesom2.0/lofoten/',False
try:
	mesh
except NameError:
    mesh = fesom_init_mesh(inputarray)
    mesh.fesom_calc_landmask()
else:
    if mesh.id!=inputarray['mesh_id']:
        mesh = fesom_init_mesh(inputarray)
        mesh.fesom_calc_landmask()
    else:
        print(' --> ___FOUND {} FESOM MESH --> will use it!___________________________'.format(mesh.id))   

 --> ___FOUND lofoten FESOM MESH --> will use it!___________________________


In [25]:
#+_____________________________________________________________________________+
#|                                                                             |
#|                         *** SET INPUT PARAMETER ***                         |
#|                                                                             |
#+_____________________________________________________________________________+
# setup variable name, runid and data path
data 			= fesom_data(inputarray) # init fesom2.0 data object
#data.descript,data.path = 'linfs' ,'/work/ollie/pscholz/results_fesom2.0/linfs/zmeanval/3/'
data.descript,data.path = 'test' ,'../results/test4nkoldunov3/'
data.var 		= 'tsurfoce'

#+_____________________________________________________________________________+
# select year to average over [start_yr, end_yr]
# data.year		= [1998,2007]
#data.year		= [1990,2000]
data.year		= [1948,1948]

# select month to average over
#data.month		= [1,2,3,4,5,6,7,8,9,10,11,12]
#data.month		= [1,2,12]
#data.month		= [6,7,8]
#data.month		= [1,2,12]
#data.month		= [3]
data.month		= []
data.record		= [1]

# select linear interpolated depth layers to average over
data.depth		= [0]
#data.depth		= np.arange(   0, 100+1,10)
# data.depth		= np.arange(   0, 200+1,10)
#data.depth		= np.arange( 200, 500+1,20)
#data.depth	    = np.arange( 500,1000+1,50)
#data.depth	    = np.arange(1000,1500+1,50)

#+_____________________________________________________________________________+
# make anomaly
#do_anomaly      = True
do_anomaly      = False
if do_anomaly==True:
    data2 			= cp.copy(data) # init fesom2.0 data object
    data2.descript,data2.path = 'kpp','/work/ollie/pscholz/results_fesom2.0/linfs/zmeanval/1/'
    
#+_____________________________________________________________________________+
#|                                                                             |
#|                         *** LOAD FVSOM DATA ***                             |
#|                                                                             |
#+_____________________________________________________________________________+
# plot topography
if data.var=='depth':
    data.value 	= mesh.nodes_2d_zg
    #data.value 	= mesh.nodes_2d_iz
    data.sname, data.lname, data.unit, data.cmap = 'depth', 'Depth', 'm', 'wbgyr'
    data.levels = np.arange(0,np.max(data.value),200)# set resolution levels
#_______________________________________________________________________________
# plot triangle resolution interpolated to node
elif data.var=='triresol':
    if len(mesh.nodes_2d_resol)==0: mesh.fesom_calc_triresol()
    data.value 	= mesh.nodes_2d_resol
    data.sname, data.lname, data.unit, data.cmap = 'triresol', 'Resolution', 'km', 'rygbw'
    data.levels = np.arange(0,np.max(data.value),10)# set resolution levels
#_______________________________________________________________________________
# plot triangle area interpolated to node
elif data.var=='triarea':
    if len(mesh.nodes_2d_area)==0: mesh.fesom_calc_triarea()
    data.value 	= mesh.nodes_2d_area
    data.sname,data.lname, data.unit, data.cmap= 'triarea', 'Area', 'km^2', 'cmocean.cm.balance'
    data.levels = np.arange(0,np.max(data.value),10)# set resolution levels
#_______________________________________________________________________________
# load all other 2d and 3d variables
else:
    fesom_load_data_horiz(mesh,data,do_output='False')
    if data.var.find('MLD')!=-1 : data.value = -data.value;
    #if data.value.size == mesh.n2dea: data.value = mesh.fesom_interp_e2n(np.array(data.value))         
    if do_anomaly==True:
        fesom_load_data_horiz(mesh,data2,do_output='False')
        if data.var.find('MLD')!=-1 : data2.value = -data2.value
        #if data2.value.size == mesh.n2dea: data2.value = mesh.fesom_interp_e2n(np.array(data2.value))
        anom = fesom_data_anom(data2,data)

#+_____________________________________________________________________________+
#|                                                                             |
#|                         *** PLOT FVSOM DATA ***                             |
#|                                                                             |
#+_____________________________________________________________________________+
# plot 2d and 2dvec data
if len(data.value2)==0:
    #___________________________________________________________________________
    # plot anomaly
    if do_anomaly==False:
        #_______________________________________________________________________
        if data.value.size == mesh.n2dea: data.value = mesh.fesom_interp_e2n(np.array(data.value))
        idxbox_e = mesh.nodes_2d_xg[mesh.elem_2d_i].max(axis=1)<inputarray['which_box'][0]
        idxbox_e = np.logical_or(idxbox_e,mesh.nodes_2d_xg[mesh.elem_2d_i].min(axis=1)>inputarray['which_box'][1])
        idxbox_e = np.logical_or(idxbox_e,mesh.nodes_2d_yg[mesh.elem_2d_i].max(axis=1)<inputarray['which_box'][2])
        idxbox_e = np.logical_or(idxbox_e,mesh.nodes_2d_yg[mesh.elem_2d_i].min(axis=1)>inputarray['which_box'][3])
        idxbox_e = idxbox_e==False # true index for triangles that are within box  
        if data.value.size ==mesh.n2dna:
            idxbox_n  = mesh.elem_2d_i[idxbox_e,:].flatten().transpose()
            idxbox_n  = np.array(idxbox_n).squeeze()
            idxbox_n  = np.unique(idxbox_n)
            idx_box   = np.zeros((mesh.n2dna,), dtype=bool)
            idx_box[idxbox_n]=True
            del idxbox_n 
            del idxbox_e
        # case of element data
        elif data.value.size ==mesh.n2dea:
            idx_box   = idxbox_e
            del idxbox_e
    
        print(data.value.shape)
        print(idx_box.shape)
        
        
        cmin,cmax = np.nanmin(data.value[idx_box]),np.nanmax(data.value[idx_box])
        cref = cmin + (cmax-cmin)/2
        cref = np.around(cref, -np.int32(np.floor(np.log10(np.abs(cref)))-1) ) 
        #cmin,cmax,cref = 1,20,10
        #cmin,cmax,cref = -500,500,0
        #cmin,cmax,cref = -0.25,-0.05,-0.15
        #cmin,cmax,cref = -1.0e-6,1.0e-6,0.0
        data.crange,data.cmap,data.cnumb = [cmin,cmax,cref],'wbgyr',40
        #data.crange,data.cmap,data.cnumb = [cmin,cmax,0.0],'blue2red',40
        fig,ax,map,cbar,hp1,tri=fesom_plot2d_data(mesh,data,figsize=[12,7],do_grid=True)
    else:
        #_______________________________________________________________________
        # interpolate elemental values to nodal vlaues
        if data.value.size  == mesh.n2dea: data.value  = mesh.fesom_interp_e2n(np.array(data.value))
        if data2.value.size == mesh.n2dea: data2.value = mesh.fesom_interp_e2n(np.array(data2.value))
        if anom.value.size  == mesh.n2dea: anom.value  = mesh.fesom_interp_e2n(np.array(anom.value))
        
        #_______________________________________________________________________
        # do common crange for line and lin2
        cmax = np.max([np.nanmax(data.value),np.nanmax(data2.value)])
        cmin = np.min([np.nanmin(data.value),np.nanmin(data2.value)])
        cref = cmin + (cmax-cmin)/2
        cref = np.around(cref, -np.int32(np.floor(np.log10(np.abs(cref)))-1) ) 
        data.crange = [cmin,cmax,cref]	
        data2.crange= [cmin,cmax,cref]
        
        #_______________________________________________________________________
        fig,ax,map,cbar,hp1,tri=fesom_plot2d_data(mesh,data,figsize=[12,7],do_grid=True)
        fig.canvas.draw()
        fig,ax,map,cbar,hp1,tri=fesom_plot2d_data(mesh,data2,figsize=[12,7],do_grid=True)
        fig.canvas.draw()   
        anom.crange= [np.nanmin(anom.value)*0.1,np.nanmax(anom.value)*0.1, 0.0]
        anom.crange=[-2,2,0]
        fig,ax,map,cbar,hp1,tri=fesom_plot2d_data(mesh,anom,figsize=[12,7])
        fig.canvas.draw()
        
else:
    
    fesom_plot2dvec_data(mesh,data,figsize=[12,7])

select single record time slice|1|
(289045,)
(289045,)

___PLOT 2D DATA____________________________________________


<IPython.core.display.Javascript object>

[cmin,cmax,cref] = [-1.8012478351593018, 10.946988105773926, 4.6]
clevel =  [ -1.9   -1.65  -1.4   -1.15  -0.9   -0.65  -0.4   -0.15   0.1    0.35
   0.6    0.85   1.1    1.35   1.6    1.85   2.1    2.35   2.6    2.85
   3.1    3.35   3.6    3.85   4.1    4.35   4.6    4.85   5.1    5.35
   5.6    5.85   6.1    6.35   6.6    6.85   7.1    7.35   7.6    7.85
   8.1    8.35   8.6    8.85   9.1    9.35   9.6    9.85  10.1   10.35
  10.6   10.85  11.1 ]


In [ ]:
print(mesh.nodes_2d_x.min())
print(mesh.nodes_2d_x.max())